# 🎥 Colab Video Compressor

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

Advanced video compression bot with GPU acceleration and multiple presets.

In [ ]:
#@title 🚀 Enhanced GPU-Accelerated Video Compression Bot

#@markdown ### 🔧 API Configuration
APP_ID = ""  #@param {type:"string"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER = "" #@param {type:"string"}
THUMBNAIL = "https://graph.org/file/75ee20ec8d8c8bba84f02.jpg" #@param {type:"string"}

#@markdown ---
#@markdown ### 📊 Compression Presets
preset_choice = "⚡ Ultra Fast (480p)" #@param ["⚡ Ultra Fast (480p)", "🏃 Fast (720p)", "⚖️ Balanced (1080p)", "💎 High Quality (1080p)", "🎬 Cinema Quality (1440p)", "🖥️ Desktop (1080p 60fps)", "📱 Mobile Optimized (720p)", "🎮 Gaming (1080p 60fps)", "🎭 Animation (1080p)", "🎥 Film Grade (4K)", "🔧 Custom"]

#@markdown ---
#@markdown ### 🛠️ Custom Compression Settings (Only used when "Custom" is selected)
custom_resolution = 720 #@param {type:"slider", min:240, max:2160, step:120}
custom_fps = 30 #@param ["24", "25", "30", "48", "50", "60", "120"] {type:"string"}
custom_quality = 26 #@param {type:"slider", min:15, max:40, step:1}
custom_audio_bitrate = "128k" #@param ["64k", "96k", "128k", "160k", "192k", "256k", "320k", "384k"]
custom_codec = "h264_nvenc" #@param ["h264_nvenc", "h265_nvenc", "libx264", "libx265"]
custom_preset = "p3" #@param ["p1", "p2", "p3", "p4", "p5", "p6"]
custom_profile = "high" #@param ["baseline", "main", "high", "main10"]

#@markdown ---
#@markdown ### 🎨 Output Settings
FILENAME_TEMPLATE = "{original_name} [{preset}]" #@param {type:"string"}
#@markdown Available placeholders: `{original_name}`, `{preset}`, `{resolution}`, `{codec}`, `{date}`, `{time}`

watermark_enabled = False #@param {type:"boolean"}
WATERMARK_TEXT = "Compressed by Bot" #@param {type:"string"}
watermark_position = "bottom-right" #@param ["top-left", "top-right", "bottom-left", "bottom-right", "center"]

#@markdown ---
#@markdown ### ⚙️ Advanced Configuration
MAX_FILE_SIZE = 4000 #@param {type:"integer"}
MAX_QUEUE_SIZE = 15 #@param {type:"integer"}
UPLOAD_CONNECTIONS = 10 #@param {type:"slider", min:1, max:20, step:1}
DEFAULT_UPLOAD_MODE = "Document" #@param ["Document", "File"]
PROGRESS_UPDATE_INTERVAL = 5 #@param {type:"integer"}
AUTO_DELETE_ORIGINAL = False #@param {type:"boolean"}
ENABLE_HARDWARE_ACCELERATION = True #@param {type:"boolean"}

import os
import subprocess
import json
import time
from datetime import datetime
from IPython.display import HTML, display, clear_output

PRESET_PARAMS = {
    "⚡ Ultra Fast (480p)": {"preset": "p1", "qp": 35, "scale": 480, "ab": "96k", "fps": 30, "codec": "h264_nvenc", "profile": "baseline", "level": "3.1", "description": "Fastest compression, smallest file size"},
    "🏃 Fast (720p)": {"preset": "p2", "qp": 28, "scale": 720, "ab": "128k", "fps": 30, "codec": "h264_nvenc", "profile": "main", "level": "3.1", "description": "Good balance of speed and quality"},
    "⚖️ Balanced (1080p)": {"preset": "p3", "qp": 26, "scale": 1080, "ab": "192k", "fps": 30, "codec": "h264_nvenc", "profile": "high", "level": "4.0", "description": "Best overall quality/size ratio"},
    "💎 High Quality (1080p)": {"preset": "p4", "qp": 22, "scale": 1080, "ab": "256k", "fps": 30, "codec": "h264_nvenc", "profile": "high", "level": "4.0", "description": "High quality with moderate file size"},
    "🎬 Cinema Quality (1440p)": {"preset": "p5", "qp": 20, "scale": 1440, "ab": "320k", "fps": 24, "codec": "h265_nvenc", "profile": "main", "level": "5.0", "description": "Cinema-grade quality with HEVC"},
    "🖥️ Desktop (1080p 60fps)": {"preset": "p3", "qp": 24, "scale": 1080, "ab": "192k", "fps": 60, "codec": "h264_nvenc", "profile": "high", "level": "4.2", "description": "Optimized for desktop/screen recordings"},
    "📱 Mobile Optimized (720p)": {"preset": "p2", "qp": 30, "scale": 720, "ab": "128k", "fps": 30, "codec": "h264_nvenc", "profile": "baseline", "level": "3.1", "description": "Optimized for mobile devices"},
    "🎮 Gaming (1080p 60fps)": {"preset": "p4", "qp": 23, "scale": 1080, "ab": "256k", "fps": 60, "codec": "h264_nvenc", "profile": "high", "level": "4.2", "description": "High quality for gaming content"},
    "🎭 Animation (1080p)": {"preset": "p5", "qp": 18, "scale": 1080, "ab": "192k", "fps": 24, "codec": "h265_nvenc", "profile": "main", "level": "4.0", "description": "Optimized for animated content"},
    "🎥 Film Grade (4K)": {"preset": "p6", "qp": 16, "scale": 2160, "ab": "384k", "fps": 24, "codec": "h265_nvenc", "profile": "main10", "level": "5.1", "description": "Professional grade 4K quality"},
    "🔧 Custom": {"preset": "custom", "qp": 0, "scale": 0, "ab": "0k", "fps": 0, "codec": "custom", "profile": "custom", "level": "custom", "description": "User-defined custom settings"}
}

def run_shell(cmd, show_output=True, is_bot=False):
    if is_bot:
        return subprocess.Popen(cmd, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    try:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True, timeout=300)
        if process.returncode != 0:
            if show_output: print(f"❌ Command failed: {cmd}\nError: {process.stderr}")
            return False, process.stderr
        if show_output and process.stdout.strip(): print(process.stdout.strip())
        return True, process.stdout
    except subprocess.TimeoutExpired:
        print(f"⏰ Command timed out: {cmd}")
        return False, "Command timed out"
    except Exception as e:
        print(f"❌ Exception running command: {str(e)}")
        return False, str(e)

def create_config_files(config):
    print("⚙️  Creating configuration files...")
    env_content = [f"{key}='{value}'" if isinstance(value, str) else f"{key}={value}" for key, value in config.items()]
    with open(".env", "w") as f: f.write("\n".join(env_content))
    print("✅ Configuration files created successfully")

def main_setup():
    try:
        if not all([APP_ID, API_HASH, BOT_TOKEN, OWNER]): raise ValueError("❌ All API credentials are required!")
        clear_output(wait=True)
        print("🚀 Enhanced Video Compression Bot Setup\n" + "="*50)
        
        print("📥 Cloning repository...")
        run_shell("git clone -q https://github.com/AdittyaMondal/ColabVideoCompressor.git /content/bot")
        os.chdir("/content/bot")
        
        print("🐍 Installing Python dependencies...")
        run_shell("pip install -q -r requirements.txt")
        
        config = create_enhanced_config()
        create_config_files(config)
        
        print("\n🎉 Setup completed successfully!")
        print("🤖 Starting bot... Check your Telegram bot! Logs will appear below.")
        print("="*50)
        
        return run_shell("python3 -m bot", is_bot=True)
    except Exception as e:
        print(f"❌ Setup failed: {str(e)}")
        return None

#@markdown ---
#@markdown ### 🚀 Run Setup
run_setup = True #@param {type:"boolean"}

bot_process = None
if run_setup:
    bot_process = main_setup()
    if not bot_process:
        print("\n🔧 Troubleshooting Tips:\n• Check your API credentials\n• Ensure your bot token is valid")


In [ ]:
#@title 📜 View Bot Logs
import threading

def stream_logs(process):
    for line in iter(process.stdout.readline, ''):
        print(line.strip())
    process.stdout.close()
    return_code = process.wait()
    if return_code:
        print(f"❌ Bot process exited with error code {return_code}")

if bot_process:
    log_thread = threading.Thread(target=stream_logs, args=(bot_process,))
    log_thread.start()
else:
    print("🤖 Bot not running. Please run the setup cell first.")